<a href="https://colab.research.google.com/github/tx1103mark/Fengshenbang-LM/blob/main/game_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
glm_merge_path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/glm4_9b_dev_result.json'
save_path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/glm4_9b_dev_result_filter.json'
save_out=open(save_path,mode='w')
with open(glm_merge_path,mode='r') as f:
    for line in f.readlines():
      temp=json.loads(line)
      temp['answer']=temp['answer'].strip()
      save_out.write(json.dumps(temp,ensure_ascii=False)+'\n')
save_out.close()

In [ ]:
glm_merge_path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/glm4_9b_dev_result_filter.json'
import json
import pandas as pd
data=[]
with open(glm_merge_path,mode='r') as f:
    for line in f.readlines():
      temp=json.loads(line)
      data.append(temp)
df=pd.DataFrame.from_dict(data)

qwen_merge_path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/qwen14b_dev.json'
qwen_data=[]
with open(qwen_merge_path,mode='r') as f:
    for line in f.readlines():
      temp=json.loads(line)
      qwen_data.append(temp)
qwen_df=pd.DataFrame.from_dict(qwen_data)


dev_path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/dev.json'
stand_answers=[]
with open(dev_path,mode='r') as f:
    json_list=json.loads(f.read())
    for json_data in json_list:
        # print(json_data)
        stand_answers.append(json_data['答案'])
df['qwen14b-answer'] = qwen_df['answer']
df['stand'] = stand_answers

df.to_excel('/opt/huawei/explorer-env/algorithm/z00421835_longbench/merge.xlsx',index=False)
df.head(20)

In [ ]:
def post_process():
    glm_merge_path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/glm4_9b_merge_222_result_filter.json'
    import json
    import pandas as pd
    data=[]
    with open(glm_merge_path,mode='r') as f:
        for line in f.readlines():
          temp=json.loads(line)
          data.append(temp)
    df=pd.DataFrame.from_dict(data)


    qwen_merge_path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/qwen14b_result_filter.json'
    qwen_data=[]
    with open(qwen_merge_path,mode='r') as f:
        for line in f.readlines():
          temp=json.loads(line)
          qwen_data.append(temp)
    qwen_df=pd.DataFrame.from_dict(qwen_data)

    other_path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/qwen_merge_glm4.json'
    other=open(other_path,mode='w')

    df['qwen_answer'] = qwen_df['answer']
    merge_answers=[]
    # print(df.head())
    for index, row in df.iterrows():
        # print(row)
        std_answer=row['answer']
        # if '无法回答' in row['qwen_answer']:
        #     print(row['ID'])
        #     std_answer=row['answer']
        if '您' in row['answer'] and '您' not in row['qwen_answer']:
            # print('---------',row['ID'])
            temp={'ID':row['ID'],'question':row['question'],'answer':row['qwen_answer']}
            other.write(json.dumps(temp,ensure_ascii=False)+'\n')
            # pass
        # if '您' in row['answer'] and '您' in row['qwen_answer']:
        #     print(row)
        if '不，' in row['qwen_answer'] and '不，' not in row['answer']:
            temp={'ID':row['ID'],'question':row['question'],'answer':row['qwen_answer']}
            other.write(json.dumps(temp,ensure_ascii=False)+'\n')
    other.close()

post_process()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import json
from transformers import set_seed
import pandas as pd
from tqdm import tqdm
set_seed(42)
import os
import numpy as np

device='cuda'
topn=40

import os
os.environ["USE_TF"] = "0"
if  os.path.exists('/home/ma-user/work/dataset/'):
    model_path='/home/ma-user/work/dataset/z30030640_data/SearchLLM-embedding/bert'
else:
    model_path='/opt/huawei/dataset/model_dir/z00421835/SearchLLM-embedding/bert'
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_path)
model.to('cuda')

def split_text_into_chunks(text):
    # 按换行符分割文本为段落
    paragraphs = text.split('\n')

    # 初始化结果列表和当前 chunk
    chunks = []
    current_chunk = ""

    for paragraph in paragraphs:
        # 去除段落两端的空格
        paragraph = paragraph.strip()

        # 如果段落为空，则跳过
        if not paragraph:
            continue

        # 检查段落开头的字符串长度
        if len(paragraph) < 5:
            # 如果当前 chunk 不为空，则将当前 chunk 添加到结果列表
            if current_chunk:
                chunks.append(current_chunk.strip())
            # 开始一个新的 chunk
            current_chunk = paragraph
        else:
            # 否则，将内容追加到当前 chunk
            current_chunk += " " + paragraph

    # 添加最后一个 chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def get_top_chunk_index(query,chunks,model,topn):
    sentences_1 = [query]
    sentences_2 = chunks
    embeddings_1 = model.encode(sentences_1, normalize_embeddings=True,batch_size=1)
    embeddings_2 = model.encode(sentences_2, normalize_embeddings=True,batch_size=1)

    similarity = embeddings_1 @ embeddings_2.T
    # print('-----similarity------',similarity)
    top_k=topn
    if len(chunks) < topn:
        top_k=len(chunks)
    index=np.argpartition(similarity[0], -top_k)[-top_k:]
    return index


def filter_item(query,item,topn):
    chunks=split_text_into_chunks(item)
    index=get_top_chunk_index(item,chunks,model,topn)
    filter_chunks=[chunks[i] for i in index]
    return '\n'.join(filter_chunks)

def glm4_model_test():
       if os.path.exists('/opt/huawei/explorer-env'):
           qwen_model_path='/opt/huawei/explorer-env/dataset/nlp_large_model_new/z00421835/model_save/glm4_test/merge-222/'
           path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/test-B-0722.json'
           out_path='/opt/huawei/explorer-env/dataset/nlp_large_data/z00421835/round1_training_data/glm4-test-B-local.json'
       else:
           qwen_model_path='/home/ma-user/work/dataset/nlp_large_model_new/z00421835/model_save/glm4_test/merge-74/'
           path='/home/ma-user/work/dataset/nlp_large_data/z00421835/round1_training_data/test-B-0722.json'
           out_path='/home/ma-user/work/dataset/nlp_large_data/z00421835/round1_training_data/glm4-test-B-local-merge-74.json'

       model = AutoModelForCausalLM.from_pretrained(
        qwen_model_path,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto"
    ).eval()
       tokenizer = AutoTokenizer.from_pretrained(qwen_model_path,trust_remote_code=True)

       examples=[]
       with open(path,mode='r') as f:
         for line in f.readlines():
            examples.append(json.loads(line))

       count = 0
       target = 300

       with open(out_path,mode='w') as out:
           for data in tqdm(examples, total=len(examples)):
                count+=1
                if count < target:
                    continue

                name=data['产品名']
                item=data['条款']
                question=data['问题']

                # item=filter_item(question,item,topn)

                prompt=f'''## 角色设定
            你是一名资深的保险顾问，擅长回答保险产品及其条款的相关问题。

            ## 任务背景与描述
               你的职责是根据【保险产品名】和【保险条款】来回答【用户问题】,直接给出答案，请注意答案的简洁和精准性。

            【保险产品名】：{name}
            【保险条款】：{item}
            【用户问题】: {question}
            '''

                inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                               add_generation_prompt=True,
                                               tokenize=True,
                                               return_tensors="pt",
                                               return_dict=True
                                               )

                inputs = inputs.to(device)

                gen_kwargs = {"max_length": 100000, "do_sample": True, "top_k": 1}
                with torch.no_grad():
                    outputs = model.generate(**inputs, **gen_kwargs)
                    outputs = outputs[:, inputs['input_ids'].shape[1]:]
                    response=tokenizer.decode(outputs[0], skip_special_tokens=True)
                    # answer=response.split('答案：')[-1]
                    print(f'---query:{question}----response:{response}---------')
                    out_dict={"ID":data['ID'],"question":question,"answer":response}
                    out.write(json.dumps(out_dict,ensure_ascii=False)+'\n')
       # out.close()


def qwen_model_infer():
    if os.path.exists('/opt/huawei/explorer-env/dataset/'):
        qwen_model_path='/opt/huawei/explorer-env/dataset/nlp_large_model_new/openLLM/Qwen2-7B-Instruct'
        # qwen_model_path='/opt/huawei/explorer-env/dataset/nlp_large_model_new/openLLM/Qwen1.5-14B-Chat'
        path='round1_training_data/test.json'
        out_path='round1_training_data/result.json'
    else:
        # qwen_model_path='/opt/huawei/dataset/model_dir/openLLM/Qwen1.5-14B-Chat'
        # qwen_model_path='/opt/huawei/dataset/model_dir/z00421835/model_save/qwen14b_mtp/merge-74'
        qwen_model_path='/opt/huawei/dataset/model_dir/openLLM/Qwen2-7B-Instruct'
        path='/opt/huawei/dataset/data_dir/z00421835/round1_training_data/test-B-0722.json'
        out_path='/opt/huawei/dataset/data_dir/z00421835/round1_training_data/qwen14b_testB_result.json'
    # qwen_model_path='/home/ma-user/work/dataset/nlp_large_model_new/openLLM/Qwen2-7B-Instruct'
    # qwen_model_path='/opt/huawei/explorer-env/dataset/nlp_large_model_new/ywx1272703/model_eval/Qwen2-7B-full_v1_11_ckpt858'
    # qwen_model_path='/opt/huawei/explorer-env/dataset/nlp_large_model_new/L1_models/Qwen2-7B-lora/l1/merge-1200/'

    # qwen_model_path='/opt/huawei/dataset/model_dir/openLLM/Qwen2-72B-Instruct'
    model = AutoModelForCausalLM.from_pretrained(
        qwen_model_path,
        torch_dtype=torch.float16,
        device_map="auto"
    ).eval()
    tokenizer = AutoTokenizer.from_pretrained(qwen_model_path)

    examples=[]
    with open(path,mode='r') as f:
         for line in f.readlines():
            examples.append(json.loads(line))
    with open(out_path,mode='w') as out:
        for data in tqdm(examples, total=len(examples)):

            name=data['产品名']
            item=data['条款']
            question=data['问题']

            item=filter_item(question,item,topn)

            prompt=f'''## 角色设定
        你是一名资深的保险顾问，擅长回答保险产品及其条款的相关问题。

        ## 任务背景与描述
          你的职责是根据【保险产品名】和【保险条款】来回答【用户问题】,直接给出答案，请注意答案的简洁和精准性。

        【保险产品名】：{name}
        【保险条款】：{item}
        【用户问题】: {question}
        '''
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]

            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            # print(text)
            model_inputs = tokenizer([text], return_tensors="pt").to(device)

            # Directly use generate() and tokenizer.decode() to get the output.
            # Use `max_new_tokens` to control the maximum output length.
            generated_ids = model.generate(
                model_inputs.input_ids,
                max_new_tokens=512
            )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]

            response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            # answer=response.split('答案：')[-1]
            print(f'---query:{question}----response:{response}---------')
            out_dict={"ID":data['ID'],"question":question,"answer":response}
            out.write(json.dumps(out_dict,ensure_ascii=False)+'\n')

def glm4_model_infer():
    if os.path.exists('/opt/huawei/explorer-env/dataset/'):
        qwen_model_path='/opt/huawei/explorer-env/dataset/nlp_large_model_new/openLLM/glm-4-9b-chat'
        # qwen_model_path='/opt/huawei/explorer-env/dataset/nlp_large_model_new/openLLM/Qwen1.5-14B-Chat'
        path='round1_training_data/test.json'
        out_path='round1_training_data/result.json'
    else:
        # qwen_model_path='/opt/huawei/dataset/model_dir/openLLM/glm-4-9b-chat'
        qwen_model_path='/opt/huawei/dataset/model_dir/z00421835/model_save/glm4_test/merge-222/'
        qwen_model_path='/opt/huawei/dataset/model_dir/z00421835/model_save/glm4_test/merge-74/'
        path='/opt/huawei/dataset/data_dir/z00421835/round1_training_data/test-B-0722.json'
        out_path=f'/opt/huawei/dataset/data_dir/z00421835/round1_training_data/glm4_9b_testB_result_merge_74_0.json'
    # qwen_model_path='/home/ma-user/work/dataset/nlp_large_model_new/openLLM/Qwen2-7B-Instruct'
    # qwen_model_path='/opt/huawei/explorer-env/dataset/nlp_large_model_new/ywx1272703/model_eval/Qwen2-7B-full_v1_11_ckpt858'
    # qwen_model_path='/opt/huawei/explorer-env/dataset/nlp_large_model_new/L1_models/Qwen2-7B-lora/l1/merge-1200/'

    # qwen_model_path='/opt/huawei/dataset/model_dir/openLLM/Qwen2-72B-Instruct'
    model = AutoModelForCausalLM.from_pretrained(
        qwen_model_path,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto"
    ).eval()
    tokenizer = AutoTokenizer.from_pretrained(qwen_model_path,trust_remote_code=True)

    # out=open(out_path,mode='w')

    # with open(path,mode='r') as f:
    #      examples=json.loads(f.read())

    examples=[]
    with open(path,mode='r') as f:
         for line in f.readlines():
            examples.append(json.loads(line))
    # count=400
    count=102
    cur=0
    with open(out_path,mode='w') as out:
        for data in tqdm(examples, total=len(examples)):
            cur +=1
            if cur < count:
                continue
            # if cur > 600:
            #     break
            # print('------cur------------',cur)
            name=data['产品名']
            item=data['条款']
            question=data['问题']

            # item=filter_item(question,item,topn)

            prompt=f'''## 角色设定
        你是一名资深的保险顾问，擅长回答保险产品及其条款的相关问题。

        ## 任务背景与描述
          你的职责是根据【保险产品名】和【保险条款】来回答【用户问题】,直接给出答案，请注意答案的简洁和精准性。

        【保险产品名】：{name}
        【保险条款】：{item}
        【用户问题】: {question}
        '''

            inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                           add_generation_prompt=True,
                                           tokenize=True,
                                           return_tensors="pt",
                                           return_dict=True
                                           )

            inputs = inputs.to(device)

            gen_kwargs = {"max_length": 128000, "do_sample": True, "top_k": 1}
            with torch.no_grad():
                outputs = model.generate(**inputs, **gen_kwargs)
                outputs = outputs[:, inputs['input_ids'].shape[1]:]
                response=tokenizer.decode(outputs[0], skip_special_tokens=True)
                # answer=response.split('答案：')[-1]
                # print(f'---query:{question}----response:{response}---------')
                out_dict={"ID":data['ID'],"question":question,"answer":response}
                out.write(json.dumps(out_dict,ensure_ascii=False)+'\n')
    # out.close()
# qwen_model_infer()
glm4_model_infer()
# glm4_model_test()


In [ ]:
#!/bin/bash

#https://github.com/modelscope/swift/blob/57ea4497c0e81c4ef7d9a6bb275366db0a12634e/docs/source/LLM/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.md
#https://github.com/modelscope/swift/blob/57ea4497c0e81c4ef7d9a6bb275366db0a12634e/docs/source/LLM/%E8%87%AA%E5%AE%9A%E4%B9%89%E4%B8%8E%E6%8B%93%E5%B1%95.md#%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E9%9B%86

echo "开始安装环境"
date
cp /opt/huawei/dataset/model_dir/z00421835/envs/swift.tar.gz /cache/
tar -xzf /cache/swift.tar.gz -C /cache/
date
echo "结束安装环境"

source /cache/bin/activate

# pip list
# pip list |grep transformers
pip install -U transformers -i http://pip.modelarts.private.com/repository/pypi/simple/ --trusted-host pip.modelarts.private.com

pip install sentence-transformers==2.7.0 -i http://pip.modelarts.private.com/repository/pypi/simple/ --trusted-host pip.modelarts.private.com

python qwen2-game-infer.py
